# 11 - Evaluating the LLM's ability to validate itself
This notebook replicates an interesting result from a [recent Microsoft paper](https://arxiv.org/pdf/2303.12712.pdf) which I believe is a reference task used to test the performance of GPT-4 over its predecessors.

Figure 1.8 in the paper showss a language model taking structured patient information, turning that into natural prose, and checking its work by verifying each of the claims made in the prose against the original facts. This result was generated using a development version of GPT-4, not the public version we have access to now.

We'll first attempt to replicate this result using earlier models, before recreating using the most modern LLM.

In [ ]:
import json

import openai

from privacy_fingerprint.common.config import (
    load_experiment_config_from_file,
    load_global_config,
    load_global_config_from_file,
)
from privacy_fingerprint.generate.language_model import LMGenerator

In [ ]:
load_experiment_config_from_file("../experiment_config.yaml")
load_global_config_from_file("../global_config.yaml")

Start by loading a pre-generated Synthea dataset.

In [ ]:
with open("/path/to/synthea_dataset.json") as fp:
    all_records = json.load(fp)
print(f"Read {len(all_records)} records")

For the sake of this experimentation we use a "random" patient.

In [ ]:
all_records[42]

## Evaluating self-assessment using text-da-vinci
Start by using the model that has proved very successful in generating convincing looking patient records from structured data - text-da-vinci, one of the 3rd generation of OpenAI models.

We will keep our existing generation prompt, and use the Microsoft prompt for validation.

In [ ]:
global_config = load_global_config()
openai.api_key = global_config.openai.api_key

COMPLETIONS_MODEL = "text-davinci-003"
example_patient_facts = all_records[42]

In [ ]:
generator = LMGenerator()
generated_records = [
    record for record in generator.generate_text([example_patient_facts])
]
print(generated_records[0])

In [ ]:
def assemble_confirmation_prompt(prose, json_format):
    """Combine the prose and structured data together with a prompt verifying the provided information."""
    return f"""Patient's facts:
{prose}

{json.dumps(json_format, indent=2)}

Please read the above medical report and verify that each claim is exactly contained in the patient's facts. Report any information which is not included in, or is missing from, the patient's facts list.
"""

In [ ]:
example_patient_prose = generated_records[0]

openai.Completion.create(
    prompt=assemble_confirmation_prompt(
        example_patient_prose, example_patient_facts
    ),
    temperature=0,
    max_tokens=300,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    model=COMPLETIONS_MODEL,
)["choices"][0]["text"].strip(" \n")

In [ ]:
print(json.dumps(example_patient_facts, indent=2))
print(example_patient_prose)

We must manually assess the model performance, as the result is returned as free text. Looking at the above information I agree with the assessment from text-da-vinci that indeed NHS number, address, DoB, visit type, doctor are indeed missing. However the model incorrectly notes gender and visit reason as missing. The latter is quite difficult, as it is easily confused with the condition being "Chronic sinusitis". 

How does this model deal with inserted infromation? I will modify the gender field, and change the visit reason in the prose.

In [ ]:
modified_patient_prose = """

Mr. Cole Monahan is a 62-year-old married female of Mixed - White and Black Caribbean ethnicity who presented to Spire Cosmetic Surgery Clare Park Hospital on June 17th, 1978 with a chief complaint of chest pain. Upon evaluation, it was determined that Mr. Monahan has a diagnosis of chronic sinusitis."""

openai.Completion.create(
    prompt=assemble_confirmation_prompt(
        modified_patient_prose, example_patient_facts
    ),
    temperature=0,
    max_tokens=300,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    model=COMPLETIONS_MODEL,
)["choices"][0]["text"].strip(" \n")

The model is still incorrect about gender, and does not note the missing visit reason.

We can attempt a different prompt to attempt 

In [ ]:
def assemble_confirmation_prompt_instruct(prose, json_format):
    """Combine the prose and structured data together with a prompt verifying the provided information."""
    return f"""Patient's facts:
{json.dumps(json_format, indent=2)}

Patient record:
{prose}

Please go through the patient facts one by one, and for each confirm whether it is present in the patient record, returning a JSON object
"""

In [ ]:
response = openai.Completion.create(
    prompt=assemble_confirmation_prompt_instruct(
        modified_patient_prose, example_patient_facts
    ),
    temperature=0,
    max_tokens=300,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    model=COMPLETIONS_MODEL,
)["choices"][0]["text"].strip(" \n")
print(response)

This is impressive, as the model has returned a valid JSON object. With templating we may achieve a more accurate result.

In [ ]:
def assemble_templated_prompt_instruct(prose, json_format):
    """Combine the prose and structured data together with a prompt verifying the provided information."""
    return f"""Patient's facts:
{json.dumps(json_format, indent=2)}

Patient record:
{prose}

Please go through the patient facts one by one, and for each confirm whether it matches the patient record, is missing from the patient record, or a modification of the patient record, returning a JSON object
using the template below:
{{
  "name": ,
  "NHS number": ,
  "address": ,
  "date of birth": ,
  "marital status": ,
  "ethnicity": ,
  "gender": ,
  "visit type": ,
  "visit date": ,
  "provider": {{
    "doctor": ,
    "facility": 
  }},
  "visit reason": ,
  "conditions": []
}}
"""

In [ ]:
response = openai.Completion.create(
    prompt=assemble_templated_prompt_instruct(
        modified_patient_prose, example_patient_facts
    ),
    temperature=0,
    max_tokens=300,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    model=COMPLETIONS_MODEL,
)["choices"][0]["text"].strip(" \n")
print(response)

This demonstrates that we can generate very valid templated data, however the output is incorrect. The NHS number, address are missing. However so are DoB, visit type, and doctor. Gender has indeed been modified, but so has the visit reason. We need greater accuracy if we are to trust in such an evaluation process. 

## Using a chat interface

In [ ]:
templating_prompt = """
Please go through the patient facts one by one, and for each confirm whether it matches the patient record, is missing from the patient record, or a modification of the patient record, returning a JSON object
using the template below:
{{
  "name": ,
  "NHS number": ,
  "address": ,
  "date of birth": ,
  "marital status": ,
  "ethnicity": ,
  "gender": ,
  "visit type": ,
  "visit date": ,
  "provider": {{
    "doctor": ,
    "facility": 
  }},
  "visit reason": ,
  "conditions": []
}}
"""


def add_to_messages(messages, user_type, message):
    messages.append({"role": user_type, "content": message})


def generate_chat_prompt(json_data, prose, validation_prompt):
    messages = []
    add_to_messages(
        messages,
        "system",
        "You are MedGPT, a helpful assistant carefully creating creating medical notes from structured data, and validating the results.",
    )
    add_to_messages(
        messages,
        "user",
        f"Describe this patient as if you were a medical doctor.\n\nPatient Facts:\n{json.dumps(json_data, indent=2)}\n",
    )
    add_to_messages(messages, "assistant", f"Patient Record:\n{prose}")
    add_to_messages(messages, "user", validation_prompt)
    return messages


for message in generate_chat_prompt(
    example_patient_facts, example_patient_prose, templating_prompt
):
    print(message["content"])

In [ ]:
CHAT_MODEL = "gpt-3.5-turbo"

response = openai.ChatCompletion.create(
    messages=generate_chat_prompt(
        example_patient_facts, example_patient_prose, templating_prompt
    ),
    model=CHAT_MODEL,
)

In [ ]:
print(response.choices[0]["message"]["content"])

The chat interface does very poorly at this task, and I'm not sure why, especially given the performance of the completion model.

### GPT-4

How does this perform in GPT-4, to follow the example provided in the Microsoft paper?

In [ ]:
openai.organization = (
    "org-XXXX"  # Replace with org that has GPT-4 access as necessary
)

response = openai.ChatCompletion.create(
    messages=generate_chat_prompt(
        example_patient_facts, example_patient_prose, templating_prompt
    ),
    model="gpt-4",
)
print(response.choices[0]["message"]["content"])

The model is correct about the missing NHS number, date of birth, visit type, and address. Let's now test the process on the prose that I have manually modified.

In [ ]:
response = openai.ChatCompletion.create(
    messages=generate_chat_prompt(
        example_patient_facts, modified_patient_prose, templating_prompt
    ),
    model="gpt-4",
)
print(response.choices[0]["message"]["content"])

In this case the model is correct about the gender being a modification, however it now believes the date of birth has been modified - technically it has given it has been replaced with an age, but this is inconsistent. 

Additionally we now have match, rather than matches, demonstrating how narrow a template must be. Finally we attempt using the original prompt as demonstrated in the Microsoft paper.

In [ ]:
response = openai.ChatCompletion.create(
    messages=assemble_templated_prompt_instruct(
        modified_patient_prose, example_patient_facts
    ),
    model="gpt-4",
)
print(response.choices[0]["message"]["content"])

The results are impressive, but will be difficult to parse at a large scale. We are limited to demo only usage of the GPT-4 model outside of projects it has been approved for, so must conclude our experiments here.